# **Multi-Label Image Classification With Tensorflow & Keras**

Image classification is one of the trending applications in machine learning. It has a wide range of applications — from facial recognition algorithms to identifying complex patterns in images like crime detections, and many other social, medical and technical applications.

In this practice session, we will play around with a simple Multi-label classification problem. We will use the power of Tensorflow and the simplicity of Keras to build a classifier that is able to categorize the images of cats and dogs and also to identify their respective breeds.

## **Getting the dataset**

Head to MachineHack, sign up and start the [Who Let The Dogs Out: Pets Breed Classification Hackathon](https://machinehack.com/hackathons/who_let_the_dogs_out_pets_breed_classification_hackathon/overview). The datasets can be downloaded on the assignment page. The training set consists of 6206 images of both cats and dogs of different breeds. We will use these images and their respective classes provided in the train.csv file to train our classifier to categorize a given image as either the image of a cat or a dog.

Multi-Label Image Classification With Tensorflow And Keras

Note:

> * Multi-label classification is a type of classification in which an object can be categorized into more than one class. For example, In the above dataset, we will classify a picture as the image of a dog or cat and also classify the same image based on the breed of the dog or cat.
> * Multi-class classification is simply classifying objects into any one of multiple categories. Such as classifying just into either a dog or cat from the dataset above.


Importing Tensorflow and Keras

In [ ]:
!python -m pip install pip --upgrade --user -q --no-warn-script-location
!python -m pip install numpy pandas seaborn matplotlib scipy statsmodels sklearn tensorflow keras opencv-python pillow scikit-image --user -q --no-warn-script-location

import IPython
IPython.Application.instance().kernel.do_shutdown(True)


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
print(tf.__version__)

Preparing the training data

To feed the images into the Neural Network we would require the images to be loaded. We are provided with a train.csv file consisting of the image names and the respective categories. We can use the sheet to load the images using the flow_from_dataframe method from Keras, but the method requires the complete filename with the extension of the image. Since we have jpg images we will format all the id’s in train.csv by adding ‘.jpg’ to all rows. We will then create a new training set with 3 columns namely Images, Animal and Breed. Since the Animal and Breed columns are categories we will convert the type to string.

In [ ]:
training_set = pd.read_csv("train.csv")

training_imgs = ["{}.jpg".format(x) for x in list(training_set.id)]

training_labels_1 = list(training_set['class_name'])
training_labels_2 = list(training_set['breed'])
training_set = pd.DataFrame( {'Images': training_imgs,'Animal': training_labels_1, 'Breed' : training_labels_2})

#Changing the type  to str
training_set.Animal = training_set.Animal.astype(str)
training_set.Breed = training_set.Breed.astype(str)

Creating a new category

One of the simplest ways to solve a Multi-label Classification problem is by converting it into  Multi-class Classification problem. Here we will combine the Animal and Breed categories to form a new set of unique categories and we will call this new feature ‘New_class’.

In [ ]:
training_set['New_class'] = training_set['Animal'] + training_set['Breed']

Let’s have a look at our new dataset.

In [ ]:
print(training_set.head())

Preprocessing Images

In [ ]:
!gdown https://drive.google.com/uc?id=1W-9crZ3f30ZDdXPZ8-wADQo33CkG8TgH

In [ ]:
!unzip image_data.zip

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_dataGen = ImageDataGenerator(rescale = 1./255,
                                  shear_range = 0.2,
                                  zoom_range = 0.2,
                                  horizontal_flip = True)

train_generator = train_dataGen.flow_from_dataframe(
                                        dataframe = training_set,
                                        directory="image_data/images_train/",x_col="Images",
                                        y_col="New_class",
                                        class_mode="categorical",
                                        target_size=(128,128),
                                        batch_size=32)

We will now preprocess the images using Keras’ ImageDataGenerator class which will convert the images into an array of vectors that can be fed to the neural network. A set of features or parameters can be initialized to the ImageDataGenerator such as rescale, shear_range, zoom_range etc. These parameters help in extracting maximum features from an image.

The flow_from_dataframe method allows us to import images from a data frame provided the path of the images using the parameter ‘directory’. The x_col specifies the independent factor which is an image and y_col represents the dependent factor which is the category of the image that we need to predict. The target size will be the size of the resulting images from the ImageDataGenerator object, batch_size is the number of sample images used to train at once.

Building Convolutional Neural Network

We are done processing the image data. Now we can proceed to build a simple convolutional neural network. Keras allows us to build neural networks effortlessly with a couple of classes and methods.

In [ ]:
classifier = Sequential()

The Sequential class initializes a network to which we can add layers and nodes.

In [ ]:
#First Convolutional layer
classifier.add(Conv2D(filters = 56,kernel_size = (3,3), activation = 'relu', input_shape = (128,128,3)))

The add method allows us to add layers of nodes to the initialized network. In the above code, we added a Convolutional layer to the network. The convolution will be performed using a 3×3 matrix as specified with the kernel _size parameter. The activation parameter sets the activation function for the nodes. The input size should be same as the size of the outputs from the ImageDataGenerator (3 is the channel width).

In [ ]:
classifier.add(MaxPooling2D(pool_size = (2,2)))

The above code adds a pooling layer to the network.

We can add as many layers as we want as shown below, however, this puts a lot of pressure on the system resources. Choose the layers and nodes based on the capability of the machines.

In [ ]:
#second Convolutional layer
classifier.add(Conv2D(32,(3,3),activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2,2)))

The line below adds a Flattening layer to the network.

In [ ]:
#Flattening
classifier.add(Flatten())

Next, we will add a hidden layer and an output layer to complete the network as done with the following code blocks.

In [ ]:
#Hidden Layer
classifier.add(Dense(units = 64, activation = 'relu'))

#Output Layer
classifier.add(Dense(units = 10 , activation = 'softmax'))

We will now compile the network to initialize the metrics, loss and weights for the network

In [ ]:
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['categorical_accuracy','accuracy'])

Let’s have a look at the description of our CNN :

In [ ]:
classifier.summary()

Training the CNN

In [ ]:
classifier.fit_generator(train_generator, epochs =50, steps_per_epoch = 60 )

The fit_generator method will train the classifier with the data we gathered by processing the images using ImageDataGenerator class. The epochs are the number of times the cycle of training repeats. Steps-per-epoch determines the number of times the weights of each node should be updated for decreasing the loss.

Predicting For Test Set

Preparing Test Data

We will prepare the test data by adding the path and file extension to the original test_set. This will help us load the images directly from the csv file using the load_img() method that you will see in the following code blocks.

In [ ]:
test_set = pd.read_csv("test.csv")
test_imgs = ["image_data/images_test/{}.jpg".format(x) for x in list(test_set.id)]
test_set = pd.DataFrame( {'Images': test_imgs })

Identifying the trained classes

The train_generator consists of the complete trained image data. Let’s have a look at the unique categories in the training data using the class_indices attribute of the train_generator. 

In [ ]:
classes = train_generator.class_indices
print(classes)

Our model will be predicting the labels in the range 0 to 9 based on the above dictionary for each category. We will need to reverse these to the original classes. We will use a reverse of the above dictionary to later convert the predictions to actual classes. The dictionary can be inverted with the following line of code:

In [ ]:
inverted_classes = dict(map(reversed, classes.items()))
print(inverted_classes)

Predicting classes

Now it’s time to load the images one by one and predict and store the category of each image from the test_set. 

In [ ]:
from keras.preprocessing import image

Y_pred = []

for i in range(len(test_set)):
  img = image.load_img(path= test_set.Images[i],target_size=(128,128,3))
  img = image.img_to_array(img)
  test_img = img.reshape((1,128,128,3))
  img_class = classifier.predict_classes(test_img)
  prediction = img_class[0]
  Y_pred.append(prediction)

The above code block loads each image from the test set preprocess it and feeds it to the classifier to predict. The predictions are stored in a list called y_pred. Notice that all the image sizes (128,128) are the same as we did for the training set, this is important and otherwise would result in an error.

Let’s take a look at the predictions:

In [ ]:
print(Y_pred)

We can see that the predictions are in the range of 0 to 9. We can now use the inverted_classes dictionary to convert the predicted labels into actual categories.

In [ ]:
prediction_classes = [ inverted_classes.get(item,item) for item in Y_pred ]

Now if we look at the prediction_classes, we will be able to see the actual categories we used to train our classifier with.

In [ ]:
print(prediction_classes)

There is one more thing to do. Remember, the original training set provided had 2 categories, class_name and breed which we later renamed as Animal and Breed. So we will now need to split the predicted categories into class_name and breed (Animal and Breed).

In [ ]:
animal = []
breed = []
for i in prediction_classes:
  animal.append(i[0]) # First character = class_name/Animal
  breed.append(i[1:]) # Last 2 characters = breed/Breed

Creating a dataframe for the predictions and writing it to an excel

In [ ]:
predictions = {}
predictions['class_name'] = animal
predictions['breed'] = breed

Let’s have a look at the predicted classes:

In [ ]:
#Writing to excel
pd.DataFrame(predictions).to_excel("predictionss.xlsx", index = False)

#**Related Articles:**

> * [Multi Class Image Classification with Tensorflow and Keras](https://analyticsindiamag.com/multi-label-image-classification-with-tensorflow-keras/)

> * [Transfer Learning in Tensorflow Keras](https://analyticsindiamag.com/a-practical-guide-to-implement-transfer-learning-in-tensorflow/)

> * [Differentiable Augmentation for Data-Efficient GAN Training](https://analyticsindiamag.com/guide-to-differentiable-augmentation-for-data-efficient-gan-training/)

> * [Guide to Albumentation](https://analyticsindiamag.com/hands-on-guide-to-albumentation/)

> * [Google STAC](https://analyticsindiamag.com/googles-stac-ssl-framework-for-object-detection/)

> * [Comparison of Transfer Learning with Multi Class Classification](https://analyticsindiamag.com/practical-comparison-of-transfer-learning-models-in-multi-class-image-classification/)
